| 项目                                                                                                                                                                           | **DDPM (随机采样)**                                                                                   | **DDIM (隐式采样)**                     |                                   |
| :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------ | :---------------------------------- | --------------------------------- |
| **目标**                                                                                                                                                                       | 采样 $x_{t-1} \sim p_\theta(x_{t-1}                                                                 | x_t)$                               | 构造确定性映射 $x_t \leftrightarrow x_0$ |
| **噪声预测模型**                                                                                                                                                                   | $\epsilon_\theta(x_t, t)$                                                                         | 相同                                  |                                   |
| **预测 $x_0$**                                                                                                                                                                 | $\hat{x}_0 = \frac{x_t - \sqrt{1-\bar{\alpha}*t},\epsilon*\theta(x_t, t)}{\sqrt{\bar{\alpha}_t}}$ | 相同                                  |                                   |
| **反推公式**                                                                                                                                                                     |                                                                                                   |                                     |                                   |
| $\displaystyle x_{t-1} = \frac{1}{\sqrt{\alpha_t}}\Big(x_t - \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}*t}}\epsilon*\theta(x_t, t)\Big) + \sigma_t z,\ \ z\sim\mathcal{N}(0, I)$ |                                                                                                   |                                     |                                   |
| $\displaystyle x_{t-1} = \sqrt{\bar{\alpha}*{t-1}}\hat{x}*0 + \sqrt{1-\bar{\alpha}*{t-1}}\Big( \sqrt{1-\eta_t^2},\epsilon*\theta(x_t, t) + \eta_t z\Big)$                    |                                                                                                   |                                     |                                   |
| **随机项**                                                                                                                                                                      | $\sigma_t z$，其中 $\sigma_t^2 = \tilde{\beta}_t$（posterior variance）                                | $\eta_t$ 控制随机性，常设 $\eta_t=0$ 得确定性采样 |                                   |
| **马尔可夫性**                                                                                                                                                                    | 马尔可夫链，每步重新采样噪声                                                                                    | 非马尔可夫链，显式确定轨迹                       |                                   |
| **特殊情况**                                                                                                                                                                     | $\sigma_t > 0$ → 随机                                                                               | $\eta_t = 0$ → 确定性（Implicit）        |                                   |
| **计算路径**                                                                                                                                                                     | $$x_t \to x_{t-1} \to \cdots \to x_0$$                                                            | $$x_T \to x_t \leftrightarrow x_0$$ |                                   |
| **采样步数**                                                                                                                                                                     | 通常 1000+                                                                                          | 10~100 步即可                          |                                   |


## 🧩 DDPM → DDIM 推导笔记 (for ipynb Markdown)

### (A) DDPM 反向生成过程

在前向扩散（加噪）过程中：

$$
q(\mathbf{x}_t \mid \mathbf{x}_0)
= \mathcal{N}\big(
\sqrt{\bar{\alpha}_t}\,\mathbf{x}_0,\;
(1-\bar{\alpha}_t)\,\mathbf{I}
\big)
$$

其中 $\alpha_t = 1 - \beta_t$，$\bar{\alpha}_t = \prod_{\tau=1}^{t} \alpha_\tau$。


反向生成过程为：


$$
p_\theta(\mathbf{x}_{t-1} \mid \mathbf{x}_t)
= \mathcal{N}\big(
\boldsymbol{\mu}_\theta(\mathbf{x}_t,t),
\sigma_t^2 \mathbf{I}
\big)
$$

其中：

$$
\boldsymbol{\mu}_\theta(\mathbf{x}_t,t)
= \frac{1}{\sqrt{\alpha_t}}
\left(
\mathbf{x}_t- \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}_t}}
  \varepsilon_\theta(\mathbf{x}_t,t)
  \right),
  \quad
  \sigma_t^2 = \beta_t.
  $$

每一步的更新为：

$$
\mathbf{x}_{t-1}
= \boldsymbol{\mu}_\theta(\mathbf{x}_t,t) + \sigma_t \mathbf{z},
  \quad \mathbf{z}\sim\mathcal{N}(\mathbf{0},\mathbf{I})
  $$

这说明 DDPM 是一个**随机马尔可夫链**，反向采样依赖噪声 $\mathbf{z}$。


### (B) Tweedie 视角：重建 $\hat{\mathbf{x}}_0$

在前向过程中：

$$
\mathbf{x}_t
= \sqrt{\bar{\alpha}_t}\,\mathbf{x}_0
+ \sqrt{1-\bar{\alpha}_t}\,\boldsymbol{\epsilon},
\quad
\boldsymbol{\epsilon} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})
$$

于是我们可以通过预测噪声 $\varepsilon_\theta(\mathbf{x}_t, t)$ 来估计干净信号：

$$
\hat{\mathbf{x}}_0(\mathbf{x}_t, t)
= \frac{1}{\sqrt{\bar{\alpha}_t}}
\left(
\mathbf{x}_t
- \sqrt{1-\bar{\alpha}_t}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\right)
\tag{1}
$$

即 Tweedie 公式的扩散版：

$$
\hat{\mathbf{x}}_0 \approx \mathbb{E}[\mathbf{x}_0 \mid \mathbf{x}_t]
$$


---

### (C) DDPM 的通用反向形式

Song 等人（2020）将 DDPM 的反向采样重写为：

$$
\mathbf{x}_{t-1}
=
\sqrt{\bar{\alpha}_{t-1}}\,
\hat{\mathbf{x}}_0(\mathbf{x}_t, t)
+
\sqrt{1-\bar{\alpha}_{t-1}-\sigma_t^2}\,
\varepsilon_\theta(\mathbf{x}_t, t)
+
\sigma_t\,\mathbf{z},
\quad
\mathbf{z} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})
\tag{2}
$$

其中 $\sigma_t$ 决定了反向过程的随机性。DDPM 中 $\sigma_t^2 = \beta_t$，因此该过程是随机的。


---

### (D) DDIM 的关键假设与推导

DDIM 的出发点是：

> 只要边缘分布 $q(\mathbf{x}_t \mid \mathbf{x}_0)$ 相同，反向过程不必是马尔可夫随机的。

因此定义新的“隐式反向过程”：

$$
q_\sigma(\mathbf{x}_{t-1} \mid \mathbf{x}_t, \mathbf{x}_0)
= \mathcal{N}\!\left(
\mathbf{x}_{t-1};
\sqrt{\bar{\alpha}_{t-1}}\,\mathbf{x}_0
+ \sqrt{1-\bar{\alpha}_{t-1}-\sigma_t^2}\,
\varepsilon_\theta(\mathbf{x}_t, t),
\sigma_t^2 \mathbf{I}
\right)
$$

若令 $\sigma_t = 0$，即不再添加随机噪声，则更新公式化为：

$$
\mathbf{x}_{t-1}
=
\sqrt{\bar{\alpha}_{t-1}}\,
\hat{\mathbf{x}}_0(\mathbf{x}_t, t)
+
\sqrt{1-\bar{\alpha}_{t-1}}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\tag{3}
$$


### (E) 代入 $\hat{\mathbf{x}}_0$ 表达式

将式 (1) 代入 (3)，得到仅依赖 $(\mathbf{x}_t, \varepsilon_\theta)$ 的更新式：

$$
\begin{aligned}
\mathbf{x}_{t-1}
&=
\sqrt{\bar{\alpha}_{t-1}}\,
\frac{1}{\sqrt{\bar{\alpha}_t}}
\left(
\mathbf{x}_t
-
\sqrt{1-\bar{\alpha}_t}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\right)
+
\sqrt{1-\bar{\alpha}_{t-1}}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\\[4pt]
&=
\sqrt{\frac{\bar{\alpha}_{t-1}}{\bar{\alpha}_t}}\,
\mathbf{x}_t
+
\left(
\sqrt{1-\bar{\alpha}_{t-1}}
-
\sqrt{\frac{\bar{\alpha}_{t-1}}{\bar{\alpha}_t}}\,
\sqrt{1-\bar{\alpha}_t}
\right)
\varepsilon_\theta(\mathbf{x}_t, t)
\end{aligned}
\tag{4}
$$

这就是 **DDIM 核心采样公式**。

---

### (F) 模型关系总结

| 模型 | 方程类型 | 随机性 | 表达式 |
|------|-----------|-----------|-----------|
| **DDPM** | 随机微分方程 (SDE) | 有噪声项 $\sigma_t \mathbf{z}$ | $$\mathbf{x}_{t-1} = \mu_\theta(\mathbf{x}_t, t) + \sigma_t \mathbf{z}$$ |
| **DDIM** | 常微分方程 (ODE) | 确定性路径 ($\sigma_t = 0$) | $$\mathbf{x}_{t-1} = \sqrt{\bar{\alpha}_{t-1}}\,\hat{\mathbf{x}}_0 + \sqrt{1-\bar{\alpha}_{t-1}}\,\varepsilon_\theta(\mathbf{x}_t, t)$$ |

因此 DDIM 可以看作是 **在保持相同边缘分布的前提下，将 DDPM 的随机 SDE 路径改写成确定性 ODE 轨迹**，从而减少采样步数、提高速度。


下面是一套你可以直接塞到附录 / 方法部分的 LaTeX 草稿，分三块：

1. 先给出 DDPM 反向过程的标准形式
2. 推出 DDIM 的确定性采样公式（完整推导步骤）
3. 给一个“DDPM vs. DDIM 采样算法”对照版伪代码环境

我会用 $x_t$ / $\varepsilon_\theta$ / $\bar\alpha_t$ 这套你现在在文里已经在用的符号，不额外换记号。

---



### (B) Tweedie 视角：先重建 $\hat{\mathbf{x}}*0$ ，再回推 $\mathbf{x}*{t-1}$

由于正向分布 $q(\mathbf{x}_t|\mathbf{x}_0)$ 是线性高斯，我们可以写出 $\mathbf{x}_t$ 与 $\mathbf{x}_0$ 的关系：
[
\mathbf{x}_t
= \sqrt{\bar{\alpha}_t},\mathbf{x}_0

* \sqrt{1-\bar{\alpha}_t},\boldsymbol{\epsilon},
  \quad
  \boldsymbol{\epsilon}\sim\mathcal{N}(\mathbf{0},\mathbf{I}).
  ]

将这个关系反过来，我们可以用网络的噪声预测 $\varepsilon_\theta(\mathbf{x}_t,t)$ 来估计“无噪版本” $\mathbf{x}_0$，即
[
\hat{\mathbf{x}}_0(\mathbf{x}_t,t)
= \frac{1}{\sqrt{\bar{\alpha}_t}}
\left(
\mathbf{x}_t

* \sqrt{1-\bar{\alpha}*t},
  \varepsilon*\theta(\mathbf{x}_t,t)
  \right).
  \tag{1}
  \label{eq:x0_hat}
  ]

式 \eqref{eq:x0_hat} 本质上是 Tweedie 公式在扩散模型里的具体化：
$\hat{\mathbf{x}}_0$ 近似 $\mathbb{E}[\mathbf{x}_0 \mid \mathbf{x}_t]$。

接下来，DDPM 的 $p_\theta(\mathbf{x}_{t-1}|\mathbf{x}*t)$ 其实也可以等价地重写成（Song et al. 2020 指出）：
[
\mathbf{x}*{t-1}
================

\sqrt{\bar{\alpha}_{t-1}},
\hat{\mathbf{x}}*0(\mathbf{x}*t,t)
+
\sqrt{1-\bar{\alpha}*{t-1} - \sigma_t^2};
\varepsilon*\theta(\mathbf{x}_t,t)
+
\sigma_t,\mathbf{z},
\quad
\mathbf{z}\sim\mathcal{N}(\mathbf{0},\mathbf{I}),
\tag{2}
\label{eq:ddpm_general}
]
其中 $\sigma_t$（方差项的自由度）在 DDPM 里被固定成 $\sigma_t^2=\beta_t$。

注意：式 \eqref{eq:ddpm_general} 跟上面“$\mu_\theta + \sigma_t z$”是同一件事，只是把它拆成了两个可解释部分：

* 一个是“往干净解 $\hat{\mathbf{x}}_0$ 回推”的项
* 一个是“保持一致噪声刻度”的项
* 加上随机高斯噪声项 $\sigma_t \mathbf{z}$

---

### (C) DDIM 的关键想法

DDIM（Denoising Diffusion Implicit Models）的核心假设是：

> 只要我们保持每个时间步的边缘分布 $q(\mathbf{x}_t \mid \mathbf{x}_0)$ 不变，我们就可以重新定义 **反向动态**，不必强制用 DDPM 的那条「随机马尔可夫链」。

换句话说：

* DDPM 的反向过程是一条随机轨迹（随机微分方程 / SDE 的离散化）。
* 但是也存在一条**确定性的轨迹**（常微分方程 / ODE 的离散化）能到达同样的边缘分布族。
* DDIM 就是显式构造这条确定性轨迹。

如何做到？
看式 \eqref{eq:ddpm_general}，如果我们允许 $\sigma_t$ 不是固定成 $\sqrt{\beta_t}$，而是把它当成“可调噪声强度”，那我们可以选择
[
\sigma_t = 0.
]

把 $\sigma_t=0$ 代入 \eqref{eq:ddpm_general}，就得到一个**完全确定性**的更新式：
[
\mathbf{x}_{t-1}
================

\sqrt{\bar{\alpha}_{t-1}},
\hat{\mathbf{x}}*0(\mathbf{x}*t,t)
+
\sqrt{1-\bar{\alpha}*{t-1}};
\varepsilon*\theta(\mathbf{x}_t,t).
\tag{3}
\label{eq:ddim_core}
]

这就是 DDIM 的采样公式。

它等价地可以展开成只依赖当前 $\mathbf{x}*t$ 与网络预测 $\varepsilon*\theta$ 的“单步映射”：
[
\begin{aligned}
\mathbf{x}*{t-1}
&=
\sqrt{\bar{\alpha}*{t-1}},
\frac{1}{\sqrt{\bar{\alpha}_t}}
\left(
\mathbf{x}_t
------------

\sqrt{1-\bar{\alpha}*t},
\varepsilon*\theta(\mathbf{x}*t,t)
\right)
+
\sqrt{1-\bar{\alpha}*{t-1}};
\varepsilon_\theta(\mathbf{x}*t,t)
[4pt]
&=
\sqrt{\frac{\bar{\alpha}*{t-1}}{\bar{\alpha}_t}},
\mathbf{x}*t
+
\left(
\sqrt{1-\bar{\alpha}*{t-1}}
---------------------------

\sqrt{\frac{\bar{\alpha}_{t-1}}{\bar{\alpha}_t}},
\sqrt{1-\bar{\alpha}*t}
\right),
\varepsilon*\theta(\mathbf{x}_t,t).
\end{aligned}
\tag{4}
\label{eq:ddim_update}
]

式 \eqref{eq:ddim_update} 正是 DDIM 论文里的离散采样更新式。
关键点有两个：

1. **去掉随机噪声项**（$\sigma_t \mathbf{z}$ 不再出现）→ 采样轨迹确定。
2. **显式地先估计 $\hat{\mathbf{x}}_0$ 再回推** → 不再是标准马尔可夫“下一个状态只看当前状态”，而是「构造一条满足相同边缘分布族的隐式逆过程」。

从连续时间角度看，DDPM 对应一个随机微分方程（SDE）的反向积分；DDIM 对应的是同一族边缘分布的常微分方程（ODE）解的离散化。DDIM 用的是后者，所以它可以用**更少的时间步**而保持相似的视觉/地球物理质量。

---

### (D) DDPM vs. DDIM 采样伪代码

```latex
\begin{algorithm}[H]
\caption{DDPM Sampling (stochastic)}
\label{alg:ddpm_sampling}
\begin{algorithmic}[1]
\Require Trained noise predictor $\varepsilon_\theta(\cdot)$; schedule $\{\alpha_t,\bar{\alpha}_t\}_{t=1}^T$
\State Sample $\mathbf{x}_T \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$
\For{$t = T, T-1, \dots, 1$}
    \State Predict noise: $\boldsymbol{\epsilon}_t \gets \varepsilon_\theta(\mathbf{x}_t, t)$
    \State Compute mean:
    \[
    \boldsymbol{\mu}_\theta(\mathbf{x}_t,t)
    =
    \frac{1}{\sqrt{\alpha_t}}
    \left(
    \mathbf{x}_t
    -
    \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}_t}}
    \boldsymbol{\epsilon}_t
    \right)
    \]
    \State Compute variance: $\sigma_t^2 \gets \beta_t$
    \If{$t > 1$}
        \State Sample noise $\mathbf{z} \sim \mathcal{N}(\mathbf{0},\mathbf{I})$
        \State $\mathbf{x}_{t-1} \gets \boldsymbol{\mu}_\theta(\mathbf{x}_t,t) + \sigma_t \mathbf{z}$
    \Else
        \State $\mathbf{x}_{t-1} \gets \boldsymbol{\mu}_\theta(\mathbf{x}_t,t)$
    \EndIf
\EndFor
\State \Return $\mathbf{x}_0$
\end{algorithmic}
\end{algorithm}
```

```latex
\begin{algorithm}[H]
\caption{DDIM Sampling (deterministic / $\sigma_t{=}0$)}
\label{alg:ddim_sampling}
\begin{algorithmic}[1]
\Require Trained noise predictor $\varepsilon_\theta(\cdot)$; schedule $\{\bar{\alpha}_t\}_{t=1}^T$
\State Sample $\mathbf{x}_T \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$
\For{$t = T, T-1, \dots, 1$}
    \State Predict noise: $\boldsymbol{\epsilon}_t \gets \varepsilon_\theta(\mathbf{x}_t, t)$
    \State Estimate clean signal (Tweedie step):
    \[
    \hat{\mathbf{x}}_0
    =
    \frac{1}{\sqrt{\bar{\alpha}_t}}
    \left(
    \mathbf{x}_t
    -
    \sqrt{1-\bar{\alpha}_t}\,
    \boldsymbol{\epsilon}_t
    \right)
    \]
    \State Deterministic update (no $\mathbf{z}$ term):
    \[
    \mathbf{x}_{t-1}
    =
    \sqrt{\bar{\alpha}_{t-1}}\,
    \hat{\mathbf{x}}_0
    +
    \sqrt{1-\bar{\alpha}_{t-1}}\,
    \boldsymbol{\epsilon}_t
    \]
\EndFor
\State \Return $\mathbf{x}_0$
\end{algorithmic}
\end{algorithm}
```

---

### (E) 你可以直接写在文中的一句解释

> 基于式 \eqref{eq:ddpm_general}，我们将 DDPM 反向一步表示成“重建 $\hat{\mathbf{x}}_0$ + 加噪回推”的形式。DDIM 通过将该式中的随机项方差 $\sigma_t^2$ 设为 $0$，从而得到式 \eqref{eq:ddim_core} 的确定性更新。这对应于用一条与 DDPM 共享同一族边缘分布 $q(\mathbf{x}_t|\mathbf{x}_0)$ 的常微分方程轨迹来生成结果，因此可以显著减少采样步数，同时保持生成质量。

---

这样你在正文里可以用 (E) 这一句话描述思想，在附录贴 (A)(B)(C)(D) 的细节。
